In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import mobilenet_v3_large
import numpy as np
import cv2
import os
import pyclipper
from shapely.geometry import Polygon

# ====================================================================================
# 1. 모델 아키텍처 정의 (학습 때 사용한 것과 완전히 동일해야 합니다)
# ====================================================================================

class MobileNetV3_Backbone(nn.Module):
    def __init__(self, pretrained=False): # 테스트 시에는 사전학습 가중치가 필요 없습니다.
        super().__init__()
        self.features = mobilenet_v3_large(weights=None).features
        self.output_indices = [3, 6, 12, 16]

    def forward(self, x):
        outputs = []
        for i, layer in enumerate(self.features):
            x = layer(x)
            if i in self.output_indices:
                outputs.append(x)
        return outputs

class DB_FPN(nn.Module):
    def __init__(self, in_channels=[24, 40, 112, 960], out_channels=256):
        super().__init__()
        self.in_convs = nn.ModuleList([nn.Conv2d(c, out_channels, 1, bias=False) for c in in_channels])
        self.out_convs = nn.ModuleList([nn.Conv2d(out_channels, out_channels // 4, 3, padding=1, bias=False) for _ in in_channels])

    def forward(self, features):
        c2, c3, c4, c5 = features
        in5 = self.in_convs[3](c5)
        in4 = self.in_convs[2](c4)
        in3 = self.in_convs[1](c3)
        in2 = self.in_convs[0](c2)

        out4 = in4 + F.interpolate(in5, size=in4.shape[2:], mode='bilinear', align_corners=False)
        out3 = in3 + F.interpolate(out4, size=in3.shape[2:], mode='bilinear', align_corners=False)
        out2 = in2 + F.interpolate(out3, size=in2.shape[2:], mode='bilinear', align_corners=False)

        p5 = F.interpolate(self.out_convs[3](in5), size=out2.shape[2:], mode='bilinear', align_corners=False)
        p4 = F.interpolate(self.out_convs[2](out4), size=out2.shape[2:], mode='bilinear', align_corners=False)
        p3 = F.interpolate(self.out_convs[1](out3), size=out2.shape[2:], mode='bilinear', align_corners=False)
        p2 = self.out_convs[0](out2)
        
        return torch.cat([p2, p3, p4, p5], dim=1)

class DB_Head(nn.Module):
    def __init__(self, in_channels=256):
        super().__init__()
        self.prob_conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 4, 3, padding=1), 
            nn.BatchNorm2d(in_channels // 4), 
            nn.ReLU(), 
            nn.ConvTranspose2d(in_channels // 4, 1, 2, 2),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.thresh_conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 4, 3, padding=1), 
            nn.BatchNorm2d(in_channels // 4), 
            nn.ReLU(), 
            nn.ConvTranspose2d(in_channels // 4, 1, 2, 2),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
    def forward(self, x): 
        return torch.sigmoid(self.prob_conv(x)), torch.sigmoid(self.thresh_conv(x))

class DBNet(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.backbone = MobileNetV3_Backbone(pretrained)
        self.fpn = DB_FPN()
        self.head = DB_Head()
    def forward(self, x): 
        return self.head(self.fpn(self.backbone(x)))

# ====================================================================================
# 2. 테스트 실행 함수
# ====================================================================================

def test_single_image(model, image_path, output_path, prob_threshold=0.5, min_area=50):
    # --- 이미지 로드 및 전처리 ---
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 한글 경로 문제 해결을 위한 이미지 로드
    img_array = np.fromfile(image_path, np.uint8)
    original_image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    if original_image is None:
        print(f"이미지를 로드할 수 없습니다: {image_path}")
        return

    h, w, _ = original_image.shape
    
    # 학습 시와 동일한 전처리 (Augmentation 제외)
    resized_image = cv2.resize(original_image, (640, 640))
    normalized_image = (resized_image / 255.0 - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])
    image_tensor = torch.from_numpy(normalized_image.transpose(2, 0, 1)).float().unsqueeze(0).to(device)

    # --- 모델 추론 ---
    model.eval()
    with torch.no_grad():
        pred_prob, _ = model(image_tensor)
    
    # --- 후처리 (확률 맵 -> 경계 상자) ---
    prob_map = pred_prob.squeeze().cpu().numpy()
    
    # 1. 확률 맵 이진화
    binary_map = (prob_map > prob_threshold).astype(np.uint8)
    
    # 2. 컨투어(Contour) 찾기
    contours, _ = cv2.findContours(binary_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 3. 컨투어 후처리 및 확장
    final_boxes = []
    for contour in contours:
        # 너무 작은 영역은 노이즈로 간주하고 무시
        if cv2.contourArea(contour) < min_area:
            continue
        
        # Pyclipper를 이용한 컨투어 확장 (DBNet의 핵심)
        polygon = Polygon(contour.squeeze(1))
        distance = polygon.area * 1.5 / polygon.length
        offset = pyclipper.PyclipperOffset()
        offset.AddPath(contour.squeeze() , pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
        expanded_contour = offset.Execute(distance)
        
        if not expanded_contour:
            continue
        
        # 최종 박스 좌표 구하기
        box = np.array(expanded_contour[0]).reshape(-1, 2)
        final_boxes.append(box)

    # --- 결과 시각화 ---
    # 원본 이미지에 경계 상자 그리기
    result_image = original_image.copy()
    for box in final_boxes:
        # 좌표를 원본 이미지 크기에 맞게 스케일링
        box[:, 0] = box[:, 0] * w / 640
        box[:, 1] = box[:, 1] * h / 640
        
        cv2.polylines(result_image, [box.astype(np.int32)], isClosed=True, color=(0, 255, 0), thickness=2)

    # 결과 이미지 저장 (한글 경로 문제 해결)
    is_success, im_buf_arr = cv2.imencode(".jpg", result_image)
    if is_success:
        im_buf_arr.tofile(output_path)
        print(f"결과 이미지가 '{output_path}'에 저장되었습니다.")


if __name__ == '__main__':
    # ====================================================================================
    # 3. 사용자 설정
    # ====================================================================================
    # 1) 학습된 모델 파일 경로
    MODEL_PATH = "dbnet_detector_epoch_20.pth" # 본인의 모델 파일명으로 변경!

    # 2) 테스트할 이미지 경로
    TEST_IMAGE_PATH = "./test03.png" # 테스트하고 싶은 이미지 경로로 변경!

    # 3) 결과 이미지를 저장할 경로
    OUTPUT_IMAGE_PATH = "./result01.jpg" # 결과 파일명으로 변경!
    # ====================================================================================
    
    # 모델 로드
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"사용 장치: {device}")
    
    model = DBNet(pretrained=False).to(device)
    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        print(f"'{MODEL_PATH}'에서 모델 가중치를 성공적으로 로드했습니다.")
    except FileNotFoundError:
        print(f"오류: 모델 파일을 찾을 수 없습니다. '{MODEL_PATH}' 경로를 확인해주세요.")
        exit()
    
    # 테스트 실행
    test_single_image(model, TEST_IMAGE_PATH, OUTPUT_IMAGE_PATH)

사용 장치: cuda
'dbnet_detector_epoch_20.pth'에서 모델 가중치를 성공적으로 로드했습니다.


C:\Users\User\AppData\Local\Temp\ipykernel_1616\3362174363.py:176: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=de

결과 이미지가 './result01.jpg'에 저장되었습니다.
